# Sentiment Analysis

Let's download the clean dataset.

In [1]:
!gdown 1YvDb54rFAs8IjLVwOstu1lNlQJ9xMY5f

Downloading...
From: https://drive.google.com/uc?id=1YvDb54rFAs8IjLVwOstu1lNlQJ9xMY5f
To: /kaggle/working/YoutubeCommentsDataSet.csv
100%|██████████████████████████████████████| 3.49M/3.49M [00:00<00:00, 25.6MB/s]


In [2]:
# Install necessary libraries
!pip install datasets transformers huggingface_hub evaluate optuna
!apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidi

In [4]:
import os
from huggingface_hub import login as hf_login
import subprocess
import wandb
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
import evaluate
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

2025-08-19 09:59:54.711108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755597594.915234      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755597594.971129      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Auto-login for Hugging Face + Weights & Biases
hf_token = os.environ.get("HF_TOKEN")
wandb_api_key = os.environ.get("WANDB_API_KEY")


hf_login(token=hf_token)
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"

wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: so-907 (so-907-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

We need to make sure that we have a GPU.

In [5]:
torch.cuda.is_available()

True

Let's start by preparing the dataset.

In [7]:
df = pd.read_csv("YoutubeCommentsDataSet.csv")
df.dropna(inplace=True)
df["Sentiment"] = df["Sentiment"].replace({"negative": 0, "neutral": 1, "positive": 2})

train_dataset, eval_dataset = train_test_split(df, random_state=42, stratify=df["Sentiment"])

# Turn into dataset object
train_dataset = Dataset.from_pandas(train_dataset)
eval_dataset = Dataset.from_pandas(eval_dataset)

# Instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", num_labels=3)

def tokenize_function(examples):
  return tokenizer(examples["Comment"],
                   padding="max_length",
                   truncation=True,
                   max_length=512
                   )

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

/tmp/ipykernel_36/1851120801.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Sentiment"] = df["Sentiment"].replace({"negative": 0, "neutral": 1, "positive": 2})


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/13773 [00:00<?, ? examples/s]

Map:   0%|          | 0/4591 [00:00<?, ? examples/s]

In [8]:
# Rename columns and define data collator
train_dataset = train_dataset.rename_column("Sentiment", "label")
eval_dataset = eval_dataset.rename_column("Sentiment", "label")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

We are going to use one of Hugging Faces's pretrained models, RoBERTa base.

In [9]:
def compute_metrics(eval_pred):
  load_accuracy = evaluate.load("accuracy")
  load_f1 = evaluate.load("f1")

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
  f1 = load_f1.compute(predictions=predictions, references=labels, average="macro")["f1"]

  return {"accuracy": accuracy, "f1": f1}

Let's setup the trainer and start training our model. We are setting some parameters to make the training faster.

In [12]:
import os

model = AutoModelForSequenceClassification.from_pretrained(
        "roberta-base",
        num_labels=3
    )

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        "roberta-base",
        num_labels=3
    )

repo_name = "sentiment-analysis-on-youtube-comments"

training_args = TrainingArguments(
   output_dir=repo_name,
   eval_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
   metric_for_best_model="f1",
   fp16=True,
   greater_is_better=True,
   report_to=["wandb"],
   run_name="roberta-base-experiments"
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=eval_dataset,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics,
   data_collator=data_collator,
   model_init=model_init
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_36/864482258.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:492: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(


Now we will use optuna to perform hyperparameter tuning.

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    n_trials=5,
    backend="optuna"
)

print(best_trial)

[I 2025-08-19 10:01:58,004] A new study created in memory with name: no-name-a1a27555-de52-4dd7-824d-e82dc5c45a1e
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.378674,0.840340,0.784874
2,0.440800,0.376626,0.853627,0.806800


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
[I 2025-08-19 10:16:01,563] Trial 0 finished with value: 1.6604268240330917 and parameters: {'learning_rate': 3.194088626974822e-05, 'num_train_epochs': 2, 'seed': 31, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 1.6604268240330917.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▂██
train/global_step,▁▂██
train/grad_norm,▁
train/learning_rate,▁
train/loss,▁


...

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
[I 2025-08-19 11:41:03,267] Trial 3 finished with value: 1.649061109129843 and parameters: {'learning_rate': 8.59325140295354e-06, 'num_train_epochs': 4, 'seed': 8, 'per_device_train_batch_size': 8}. Best is trial 2 with value: 1.6613740069011005.
Some weights of DistilBertForSequenceClassification were not initialized from

eval/accuracy,▁▆█▇
eval/f1,▁▆█▆
eval/loss,▄▁▅█
eval/runtime,▄▁█▃
eval/samples_per_second,▅█▁▆
eval/steps_per_second,▅█▁▆
train/epoch,▁▂▂▃▄▅▆▆▇██
train/global_step,▁▂▂▃▄▅▆▆▇██
train/grad_norm,▁▁▅█▆▅
train/learning_rate,█▇▅▄▂▁
train/loss,█▅▃▃▂▁


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.517900,0.394717,0.842736,0.788943


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
